# Filter background from D&F images.
We have combined DAPI (blue) and FLUORO (green) images.  
Here, partition each image into patches.  
Filter the background (mostly-black) patches.

Resources:  

In [1]:
import os
import glob
import cv2 # OpenCV-Python
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import numpy as np
import json

DIR_IMAGES_RAW = "/home/jrm/Martinez/images/raw/"
PATTERN_IMAGES_RAW = "*.DF1.*.tif"
PATCH_SIZE=224  # matches VGG

### pixel_list = pixel_array.tolist()  # slow !

In [2]:
def get_image_names(path,pattern):
    #RAW_IMAGE_NAMES = os.listdir(DIR_IMAGES_RAW)
    paths = glob.glob(path+pattern)
    names = [os.path.basename(x) for x in paths]
    return names
FILENAMES_IMAGES_RAW = get_image_names(DIR_IMAGES_RAW,PATTERN_IMAGES_RAW)

In [3]:
def load_pixel_array(path,filename,verbose=False):
    im = Image.open(path+filename)
    ima = np.array(im)   # convert to numpy
    if verbose:
        print(filename, ima.size, ima.shape)
    return ima

In [4]:
print(FILENAMES_IMAGES_RAW)

['F15.DF1.135.tif', 'D5.DF1.43.tif', 'H3.DF1.27.tif', 'C11.DF1.96.tif', 'H13.DF1.114.tif', 'I1.DF1.09.tif', 'G3.DF1.25.tif', 'C1.DF1.01.tif', 'F11.DF1.94.tif', 'F9.DF1.80.tif', 'H7.DF1.62.tif', 'A3.DF1.26.tif', 'A5.DF1.45.tif', 'H15.DF1.132.tif', 'D1.DF1.02.tif', 'G15.DF1.133.tif', 'B15.DF1.134.tif', 'I5.DF1.48.tif', 'E9.DF1.75.tif', 'I13.DF1.117.tif', 'F3.DF1.22.tif', 'B7.DF1.60.tif', 'B13.DF1.115.tif', 'E7.DF1.64.tif', 'F7.DF1.63.tif', 'D3.DF1.20.tif', 'E5.DF1.42.tif', 'H1.DF1.07.tif']


In [5]:
pixel_array = load_pixel_array(DIR_IMAGES_RAW,'H1.DF1.07.tif',True)
avg = np.average(pixel_array,axis=(0,1))
std = np.std(pixel_array,axis=(0,1))
print("Average",avg,"STD",std)
print(pixel_array[0,0])

H1.DF1.07.tif 479598264 (12513, 12776, 3)
Average [ 0.         40.01907205 28.02747956] STD [ 0.         25.25630313 43.83137465]
[ 0 35  2]


In [18]:
background=np.array( [20,20,20] )
bright=np.array( [0,40,40] )
width = pixel_array.shape[0]
height = pixel_array.shape[1]
w=0
h=0
back_patches = 0
fore_patches = 0
back_example = None
fore_example = None
threshold=PATCH_SIZE*PATCH_SIZE/2
while h+PATCH_SIZE<=height:
    w=0
    while w+PATCH_SIZE<=width:
        patch = pixel_array[w:w+PATCH_SIZE,h:h+PATCH_SIZE]
        dark_pixels = np.sum(np.all(patch<background,axis=2))
        if dark_pixels>=threshold:
            back_patches += 1
            back_example = patch
        else:
            bright_pixels = np.sum(np.all(patch>=bright,axis=2))
            if bright_pixels > threshold:
                fore_example = patch
            fore_patches += 1
        w+=PATCH_SIZE
    h+=PATCH_SIZE
print("back=",back_patches,"fore=",fore_patches)

back= 356 fore= 2779


In [19]:
back_image = Image.fromarray(back_example)
fore_image = Image.fromarray(fore_example)
back_image.save('back.tif')
fore_image.save('fore.tif')